In [1]:
import pandas as pd
import numpy as np

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [4]:
class finominaStock():
    def __init__(self):
        self.chrome_options = webdriver.ChromeOptions()
        self.driver = webdriver.Chrome(executable_path = "./webdriver/chromedriver.exe",chrome_options=self.chrome_options)
        self.driver.set_page_load_timeout(60)
        self.start_symbol = ""
        
    def run(self):
        if self.start_symbol == "":
            start = True
        else:
            start = False
        ####################################
        df = pd.read_excel("stockList.xlsx")
        for i,r in df.iterrows():
            if r['Symbol'] == self.start_symbol:
                start = True
            
            if start :
                print(f"{r['Symbol']} [running]",end="\r")
                try:
                    self.driver.get(f"https://www.finnomena.com/stock/{r['Symbol']}")
                    WebDriverWait(self.driver, 0,10).until(EC.visibility_of_element_located((By.ID, "Close")))
                except:
                    print(f"{r['Symbol']} [error]   ")
                growthrate = self.financial_ratio("stock-growthrate")
                ratio = self.financial_ratio("stock-financial-ratio")
                statistics = self.financial_ratio("stock-statistics")
                #financial_report = self.financial_ratio("stock-financial-report")
                
               

                data = pd.concat([growthrate, ratio, statistics],axis=1, join='outer')
                data.drop(data.tail(1).index,inplace=True) 
                
                data['Symbol'] = r['Symbol']
                data['Market'] = r['Market']
                data['IndusGroup'] = r['IndusGroup']
                data['BusinessGroup'] = r['BusinessGroup']
                
                data.to_csv(f"./data/{r['Symbol']}.csv",mode='w',encoding='utf-8')
                print(f"{r['Symbol']} [done]   ")
            
    
    #financial-ratio
    def financial_ratio( self,Id ):
        topic = self.driver.find_element(By.ID, Id).find_element(By.CLASS_NAME, 'topic-wrapper').text.split("\n")
        del topic[0]

        content = self.driver.find_element(By.ID, Id).find_element(By.CLASS_NAME, 'content')
        date = content.find_element(By.CLASS_NAME, 'timeline-wrapper').text

        data = {}
        for i,r in enumerate( content.find_elements(By.CLASS_NAME,"data-wrapper") ):
            data[f"{topic[i]}"] = []
            for c in r.find_elements(By.CLASS_NAME,"data-each"):
                data[f"{topic[i]}"] += [c.text]

        data = pd.DataFrame(data,index=date.split("\n"))
        #data["Date"] = date.split("\n")
        return data

In [5]:
finomina = finominaStock()
finomina.start_symbol = 'WP'
finomina.run()

SMPC [done]   
SMT [done]   
SNC [done]   
SNNP [done]   
SNP [done]   
SO [done]   
SOLAR [done]   
SONIC [done]   
SORKON [done]   
SPA [done]   
SPACK [done]   
SPALI [done]   
SPC [done]   
SPCG [done]   
SPG [done]   
SPI [done]   
SPRC [done]   
SPRIME [done]   
SPVI [done]   
SQ [done]   
SR [done]   
SRICHA [done]   
SRIPANWA [done]   
SSC [done]   
SSF [done]   
SSP [done]   
SSPF [done]   
SSS [done]   
SSSC [done]   
SST [done]   
SSTRT [done]   
STA [done]   
STANLY [done]   
STARK [done]   
STC [done]   
STEC [done]   
STECH [done]   
STGT [done]   
STHAI [done]   
STI [done]   
STOWER [done]   
STP [done]   
STPI [done]   
SUC [done]   
SUN [done]   
SUPER [done]   
SUPEREIF [done]   
SUSCO [done]   
SUTHA [done]   
SVH [done]   
SVI [done]   
SVOA [done]   
SVT [done]   
SWC [done]   
SYMC [done]   
SYNEX [done]   
SYNTEC [done]   
TACC [done]   
TAE [done]   
TAKUNI [done]   
TAPAC [done]   
TASCO [done]   
TC [done]   
TCAP [done]   
TCC [done]   
TCCC [done]   
TCJ [d

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="stock-growthrate"]"}
  (Session info: chrome=107.0.5304.108)
Stacktrace:
Backtrace:
	Ordinal0 [0x00D2ACD3+2075859]
	Ordinal0 [0x00CBEE61+1633889]
	Ordinal0 [0x00BBB7BD+571325]
	Ordinal0 [0x00BEAC2F+764975]
	Ordinal0 [0x00BEAE1B+765467]
	Ordinal0 [0x00C1D0F2+970994]
	Ordinal0 [0x00C07364+881508]
	Ordinal0 [0x00C1B56A+963946]
	Ordinal0 [0x00C07136+880950]
	Ordinal0 [0x00BDFEFD+720637]
	Ordinal0 [0x00BE0F3F+724799]
	GetHandleVerifier [0x00FDEED2+2769538]
	GetHandleVerifier [0x00FD0D95+2711877]
	GetHandleVerifier [0x00DBA03A+521194]
	GetHandleVerifier [0x00DB8DA0+516432]
	Ordinal0 [0x00CC682C+1665068]
	Ordinal0 [0x00CCB128+1683752]
	Ordinal0 [0x00CCB215+1683989]
	Ordinal0 [0x00CD6484+1729668]
	BaseThreadInitThunk [0x758B7BA9+25]
	RtlInitializeExceptionChain [0x76FEBB9B+107]
	RtlClearBits [0x76FEBB1F+191]
